In [1]:
import pandas as pd
import os
import glob
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
from skfda.representation.grid import FDataGrid
import fdasrsf as fs


import skfda
from skfda.datasets import fetch_growth
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import (
    BSplineBasis,
    FourierBasis,
    MonomialBasis,
)


from __future__ import annotations

from typing import Any, Mapping, Tuple

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import sklearn.cluster
from cartopy.io.img_tiles import GoogleTiles
from matplotlib.axes import Axes
from matplotlib.figure import Figure

from skfda.datasets import fetch_aemet
from skfda.exploratory.depth import ModifiedBandDepth
from skfda.exploratory.visualization import Boxplot, MagnitudeShapePlot
from skfda.exploratory.visualization.fpca import FPCAPlot
from skfda.misc.metrics import l2_distance
from skfda.ml.clustering import KMeans
from skfda.preprocessing.dim_reduction import FPCA

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from kneed import KneeLocator


from sklearn.datasets import fetch_openml
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import CondensedNearestNeighbour, TomekLinks
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from IPython.display import Image

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Data Cleaning and Time Warping

In [2]:
#read each csv file in current_directory and store it in dataframes

current_directory = os.getcwd()
csv_files = glob.glob(os.path.join(current_directory, 'ABR_exported_files/*.csv'))

dataframes = {}
for file in csv_files:
    filename = os.path.splitext(os.path.basename(file))[0]
    if pd.read_csv(file).shape[1] > 1:
        dataframes[filename] = pd.read_csv(file)
    else:
        dataframes[filename] = pd.read_csv(file, skiprows=2)

In [3]:
subjects = [0,1,2,3,4,5,6,7,8,9,10]
frequencies = [4000, 8000, 16000, 24000, 32000]
dB_levels = [20,25,30,35,40,45,50,55,60,65,70,75,80,85,90]
results_df = pd.DataFrame()

# Iterate over all subject-frequency pairs
for subject in subjects:
    for frequency in frequencies:
        waves = []
        available_dB_levels = []  # List to store the available dB levels for this subject/frequency pairing
        for dB in dB_levels:
            filename = list(dataframes.items())[subject][0]
            df = list(dataframes.values())[subject]
            khz = df[df['Freq(Hz)'] == frequency]
            dbkhz = khz[khz['Level(dB)'] == dB]  # Select the dB level
            if not dbkhz.empty:
                index = dbkhz.index.values[0]
                final = df.iloc[index,48:]
                final = pd.to_numeric(final, errors='coerce')
                wave = final.tolist()[:-1]
                waves.append(wave)
                available_dB_levels.append(dB)  # Add the dB level to the list of available dB levels

        # Align the waves using dynamic time warping
        np_waves = np.array(waves)
        time = np.linspace(0, 1, np_waves.shape[1])
        obj = fs.fdawarp(np_waves.T, time)
        obj.srsf_align(parallel=True)
        aligned_waves = obj.fn.T

        # Iterate over all available dB levels to find the peaks in the aligned waves
        for i, dB in enumerate(available_dB_levels):
            wave = aligned_waves[i]
            wave_after_30 = wave[50:-30]  # start from the 30th point and end before the last 30 points
            peaks, _ = find_peaks(wave_after_30, distance=15)
            peaks = [peak + 50 for peak in peaks]  # adjust the indices
            highest_peaks_indices = sorted(peaks, key=lambda x: wave[x], reverse=True)[:5]
            highest_peaks_indices.sort()
            peak_coordinates = []
            for peak in highest_peaks_indices:
                peak_coordinates.extend([peak, wave[peak]])

            # If fewer than 5 peaks, fill the remaining coordinates with None
            while len(peak_coordinates) < 10:
                peak_coordinates.extend([None, None])

            # Add the results to the dataframe
            new_row = pd.DataFrame([{'Subject': filename, 'Frequency': frequency, 'dB': dB, 'Peak1_X': peak_coordinates[0], 'Peak1_Y': peak_coordinates[1], 'Peak2_X': peak_coordinates[2], 'Peak2_Y': peak_coordinates[3], 'Peak3_X': peak_coordinates[4], 'Peak3_Y': peak_coordinates[5], 'Peak4_X': peak_coordinates[6], 'Peak4_Y': peak_coordinates[7], 'Peak5_X': peak_coordinates[8], 'Peak5_Y': peak_coordinates[9]}])
            results_df = pd.concat([results_df, new_row], ignore_index=True)


Initializing...


/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas re

Compute Karcher Mean of 8 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 14 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
I

/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T/ipykernel_11530/2423822563.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 8 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterati

/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homeb

Compute Karcher Mean of 15 function in SRSF space with lam=0
updating step: r=1


/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/time_warping.py:210: RuntimeWarning: invalid value encountered in divide
  vtil[:,k] = v/d
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/time_warping.py:211: RuntimeWarning: divide by zero encountered in scalar divide
  dtil[k] = 1.0/d
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/time_warping.py:230: RuntimeWarning: divide by zero encountered in scalar divide
  qun[r] = norm(mq[:, r + 1] - mq[:, r]) / norm(mq[:, r])


updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
upd

In [4]:
# remove rows with NA
results_df = results_df[~results_df['Subject'].str.startswith('A')]

#add deaf column
def check_subject(subject):
    if subject.startswith(('B1_1285', 'B2_1291', 'B2_1290')):
        return 1
    else:
        return 0

results_df['Deaf'] = results_df['Subject'].apply(check_subject)

In [5]:
results_df

,Subject,Frequency,dB,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf
0,B1_1282_baseline,4000,55,81,0.001650,112,0.001422,136,0.001335,152,-0.000050,180,0.001415,0
1,B1_1282_baseline,4000,60,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0
2,B1_1282_baseline,4000,65,81,0.007780,112,-0.001040,136,0.003537,151,0.000362,191,0.000339,0
3,B1_1282_baseline,4000,70,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0
4,B1_1282_baseline,4000,75,81,0.007113,112,0.002525,136,0.006475,151,0.005110,191,0.009667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,32000,70,79,0.009000,100,0.006000,117,0.019526,169,0.008000,201,0.010877,0
506,B2_1292_baseline,32000,75,79,0.008000,117,0.027383,139,0.007000,169,0.010000,201,0.011000,0
507,B2_1292_baseline,32000,80,79,0.013000,117,0.030194,139,0.009000,169,0.013000,200,0.006000,0
508,B2_1292_baseline,32000,85,58,0.013000,79,0.010292,117,0.031957,169,0.015000,201,0.016877,0


In [6]:
# Define the dB thresholds for each subject and frequency
thresholds = {
    'B1': {
        4000: {'1284': 70, '1285': 90, '1286': 80, '1287': 70, '1279': 70, '1280': 80, '1281': 90, '1282': 75},
        8000: {'1284': 45, '1285': 70, '1286': 50, '1287': 30, '1279': 30, '1280': 50, '1281': 65, '1282': 50},
        16000: {'1284': 45, '1285': 60, '1286': 35, '1287': 20, '1279': 25, '1280': 35, '1281': 50, '1282': 35},
        24000: {'1284': 45, '1285': 65, '1286': 40, '1287': 35, '1279': 35, '1280': 40, '1281': 55, '1282': 40},
        32000: {'1284': 50, '1285': 60, '1286': 35, '1287': 30, '1279': 35, '1280': 35, '1281': 70, '1282': 45}
    },
    'B2': {
        4000: {'1283': 70, '1291': 90, '1292': 80, '1290': 80, '1288': 60, '1289': 80},
        8000: {'1283': 30, '1291': 85, '1292': 60, '1290': 75, '1288': 45, '1289': 40},
        16000: {'1283': 20, '1291': 60, '1292': 40, '1290': 60, '1288': 30, '1289': 25},
        24000: {'1283': 35, '1291': 65, '1292': 40, '1290': 70, '1288': 35, '1289': 35},
        32000: {'1283': 35, '1291': 75, '1292': 35, '1290': 90, '1288': 50, '1289': 35}
    }
}

# Add the 'signal' column
def check_signal(row):
    group = row['Subject'].split('_')[0]
    subject = row['Subject'].split('_')[1]
    frequency = row['Frequency']
    dB = row['dB']
    
    # Check if any of the values are nan
    if pd.isnull(group) or pd.isnull(subject) or pd.isnull(frequency) or pd.isnull(dB):
        return np.nan  # or return some other value that makes sense in your context
    
    if dB >= thresholds[group][frequency][subject]:
        return 1
    else:
        return 0


results_df['Signal'] = results_df.apply(check_signal, axis=1)

results_df.head()


,Subject,Frequency,dB,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Signal
0,B1_1282_baseline,4000,55,81,0.001650,112,0.001422,136,0.001335,152,-0.000050,180,0.001415,0,0
1,B1_1282_baseline,4000,60,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0,0
2,B1_1282_baseline,4000,65,81,0.007780,112,-0.001040,136,0.003537,151,0.000362,191,0.000339,0,0
3,B1_1282_baseline,4000,70,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0,0
4,B1_1282_baseline,4000,75,81,0.007113,112,0.002525,136,0.006475,151,0.005110,191,0.009667,0,1


In [7]:
encoded = pd.get_dummies(results_df, columns=['Frequency'])
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoded['dB_encoded'] = encoder.fit_transform(encoded[['dB']])
encoded = encoded.drop('dB', axis=1)


In [8]:
encoded

,Subject,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Signal,Frequency_4000,Frequency_8000,Frequency_16000,Frequency_24000,Frequency_32000,dB_encoded
0,B1_1282_baseline,81,0.001650,112,0.001422,136,0.001335,152,-0.000050,180,0.001415,0,0,True,False,False,False,False,7.0
1,B1_1282_baseline,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0,0,True,False,False,False,False,8.0
2,B1_1282_baseline,81,0.007780,112,-0.001040,136,0.003537,151,0.000362,191,0.000339,0,0,True,False,False,False,False,9.0
3,B1_1282_baseline,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0,0,True,False,False,False,False,10.0
4,B1_1282_baseline,81,0.007113,112,0.002525,136,0.006475,151,0.005110,191,0.009667,0,1,True,False,False,False,False,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,79,0.009000,100,0.006000,117,0.019526,169,0.008000,201,0.010877,0,1,False,False,False,False,True,10.0
506,B2_1292_baseline,79,0.008000,117,0.027383,139,0.007000,169,0.010000,201,0.011000,0,1,False,False,False,False,True,11.0
507,B2_1292_baseline,79,0.013000,117,0.030194,139,0.009000,169,0.013000,200,0.006000,0,1,False,False,False,False,True,12.0
508,B2_1292_baseline,58,0.013000,79,0.010292,117,0.031957,169,0.015000,201,0.016877,0,1,False,False,False,False,True,13.0


# Train/Test Split

In [9]:
#train/test split
from sklearn.model_selection import train_test_split
encoded = encoded.dropna()

feature_cols = [col for col in encoded.columns if col not in ['Signal']]
X = encoded[feature_cols]
y = encoded['Signal']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)


# R Wrapper Setup

In [10]:
import rpy2
import rpy2.robjects as robjects
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.ipython.ggplot import image_png
from rpy2.robjects.packages import importr, data
utils = importr('utils')
base = importr('base')
utils.install_packages('stats')
utils.install_packages('lme4')


/opt/homebrew/anaconda3/lib/python3.11/site-packages/rpy2/robjects/lib/ggplot2.py:70: UserWarning: This was designed against ggplot2 versions starting with 3.3. but you have 3.4.4
  warnings.warn(


--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Austria [https]
 6: Belgium (Brussels) [https]
 7: Brazil (PR) [https]
 8: Brazil (RJ) [https]
 9: Brazil (SP 1) [https]
10: Brazil (SP 2) [https]
11: Bulgaria [https]
12: Canada (MB) [https]
13: Canada (ON) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Guangzhou) [https]
20: China (Jinan) [https]
21: China (Lanzhou) [https]
22: China (Nanjing) [https]
23: China (Shanghai 2) [https]
24: China (Shenzhen) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Czech Republic [https]
28: Denmark [https]
29: Ecuador (Cuenca) [https]
30: France (Lyon 1) [https]
31: France (Lyon 2) [https]
32: France (Marseille) [https]
33: France (Paris 1) [https]
34: Germany (

R[write to console]: trying URL 'https://ftp.osuosl.org/pub/cran/bin/macosx/big-sur-arm64/contrib/4.3/lme4_1.1-35.1.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 6848176 bytes (6.5 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 


The downloaded binary packages are in
	/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T//RtmpcKOUEV/downloaded_packages


<rpy2.rinterface_lib.sexp.NULLType object at 0x171b0d210> [0]

In [11]:
stats = importr('stats')
lme4 = importr('lme4')
import rpy2.robjects.lib.ggplot2 as ggplot2

In [13]:
utils.install_packages('Matrix')

R[write to console]: trying URL 'https://ftp.osuosl.org/pub/cran/bin/macosx/big-sur-arm64/contrib/4.3/Matrix_1.6-5.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 5355734 bytes (5.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to c


The downloaded binary packages are in
	/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T//RtmpcKOUEV/downloaded_packages


<rpy2.rinterface_lib.sexp.NULLType object at 0x171b0d210> [0]

In [19]:
utils.install_packages('lme4', type='source')

R[write to console]: also installing the dependency ‘Matrix’


R[write to console]: trying URL 'https://ftp.osuosl.org/pub/cran/src/contrib/Matrix_1.6-5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2883851 bytes (2.8 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to consol

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c Csparse.c -o Csparse.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c abIndex.c -o abIndex.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c attrib.c -o attrib.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c bind.c -o bind.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g

In file included from ../Core/cholmod_change_factor.c:113:
../Core/t_cholmod_change_factor.c:37:18: warning: variable 'Lz' set but not used [-Wunused-but-set-variable]
    double *Lx, *Lz ;
                 ^
In file included from ../Core/cholmod_change_factor.c:115:
../Core/t_cholmod_change_factor.c:37:18: warning: variable 'Lz' set but not used [-Wunused-but-set-variable]
    double *Lx, *Lz ;
                 ^
../Core/cholmod_change_factor.c:262:17: warning: variable 'xsize' set but not used [-Wunused-but-set-variable]
    Int nsuper, xsize, ssize ;
                ^
../Core/cholmod_change_factor.c:525:21: warning: implicit conversion from 'size_t' (aka 'unsigned long') to 'double' changes value from 18446744073709551615 to 18446744073709551616 [-Wimplicit-const-int-float-conversion]
                xlnz = MIN (xlnz, Size_max) ;
                       ~~~~~~~~~~~^~~~~~~~~
../Include/cholmod_internal.h:207:19: note: expanded from macro 'Size_max'
#define Size_max ((size_t) (-1))
   

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c ../Core/cholmod_common.c -o cholmod_common.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c ../Core/cholmod_complex.c -o cholmod_complex.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c ../Core/cholmod_copy.c -o cholmod_copy.o
clang -arch arm64 -I"/Library/Framew

../Core/cholmod_copy.c:275:25: warning: variable 'lo' set but not used [-Wunused-but-set-variable]
    Int nrow, ncol, up, lo, values, diag, astype ;
                        ^
../Core/cholmod_copy.c:275:21: warning: variable 'up' set but not used [-Wunused-but-set-variable]
    Int nrow, ncol, up, lo, values, diag, astype ;
                    ^
2 warnings generated.
In file included from ../Core/cholmod_dense.c:45:
../Core/t_cholmod_dense.c:27:23: warning: variable 'Az' set but not used [-Wunused-but-set-variable]
    double *Ax, *Xx, *Az, *Xz ;
                      ^
../Core/t_cholmod_dense.c:27:13: warning: variable 'Ax' set but not used [-Wunused-but-set-variable]
    double *Ax, *Xx, *Az, *Xz ;
            ^
../Core/t_cholmod_dense.c:27:28: warning: variable 'Xz' set but not used [-Wunused-but-set-variable]
    double *Ax, *Xx, *Az, *Xz ;
                           ^
In file included from ../Core/cholmod_dense.c:47:
../Core/t_cholmod_dense.c:27:23: warning: variable 'Az' set but 

...

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -DDLONG -c  ../Supernodal/cholmod_super_solve.c -o cholmod_l_super_solve.o


4 warnings generated.
../Supernodal/cholmod_super_symbolic.c:562:7: warning: variable 'totsize' set but not used [-Wunused-but-set-variable]
                Int totsize ;
                    ^
../Supernodal/cholmod_super_symbolic.c:182:17: warning: unused variable 'env_max_bytes' [-Wunused-variable]
    const char* env_max_bytes;
                ^
../Supernodal/cholmod_super_symbolic.c:181:17: warning: unused variable 'env_use_gpu' [-Wunused-variable]
    const char* env_use_gpu;
                ^
../Supernodal/cholmod_super_symbolic.c:183:12: warning: unused variable 'max_bytes' [-Wunused-variable]
    size_t max_bytes;
           ^
../Supernodal/cholmod_super_symbolic.c:185:12: warning: unused variable 'max_fraction' [-Wunused-variable]
    double max_fraction;
           ^
../Supernodal/cholmod_super_symbolic.c:184:17: warning: unused variable 'env_max_fraction' [-Wunused-variable]
    const char* env_max_fraction;
                ^
../Supernodal/cholmod_super_symbolic.c:577:24: war

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -DDLONG -c  ../Supernodal/cholmod_super_symbolic.c -o cholmod_l_super_symbolic.o
ar -rucs ../../CHOLMOD.a cholmod_aat.o cholmod_add.o cholmod_band.o cholmod_change_factor.o cholmod_common.o cholmod_complex.o cholmod_copy.o cholmod_dense.o cholmod_error.o cholmod_factor.o cholmod_memory.o cholmod_sparse.o cholmod_transpose.o cholmod_triplet.o cholmod_version.o cholmod_check.o cholmod_read.o cholmod_write.o cholmod_amd.o cholmod_analyze.o cholmod_colamd.o cholmod_etree.o cholmod_factorize.o cholmod_postorder.o cholmod_rcond.o cholmod_resymbol.o cholmod_rowcolcounts.o cholmod_rowfac.o cholmod_solve.o cholmod_spsolve.o cholmod_drop.o cholmod_horzcat.o cholmod_norm.o cholmod_scale.o cholmod_sdmult.o cholmod_ssmult.o cholm

ld: warning: directory not found for option '-L/opt/gfortran/lib/gcc/aarch64-apple-darwin20.0/12.2.0'
ld: warning: directory not found for option '-L/opt/gfortran/lib'
ld: library not found for -lgfortran
clang-14: error: linker command failed with exit code 1 (use -v to see invocation)
make: *** [/Library/Frameworks/R.framework/Resources/share/make/shlib.mk:10: Matrix.so] Error 1
ERROR: compilation failed for package ‘Matrix’
* removing ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library/Matrix’
ERROR: dependency ‘Matrix’ is not available for package ‘lme4’
* removing ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library/lme4’
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/private/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T/RtmpcKOUEV/downloaded_packages’
R[write to console]: 
R[write to console]: 



<rpy2.rinterface_lib.sexp.NULLType object at 0x171b0d210> [0]

In [18]:
# remove packages
utils.remove_packages('Matrix')
utils.remove_packages('lme4')

# install lme4 package from source
utils.install_packages('lme4', type='source')

# import lme4 package
lme4 = importr('lme4')

R[write to console]: Removing package from ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library’
(as ‘lib’ is unspecified)

R[write to console]: Removing package from ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependency ‘Matrix’


R[write to console]: trying URL 'https://ftp.osuosl.org/pub/cran/src/contrib/Matrix_1.6-5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2883851 bytes (2.8 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c Csparse.c -o Csparse.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c abIndex.c -o abIndex.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c attrib.c -o attrib.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c bind.c -o bind.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I./SuiteSparse_config -DNTIMER  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g

In file included from ../Core/cholmod_change_factor.c:113:
../Core/t_cholmod_change_factor.c:37:18: warning: variable 'Lz' set but not used [-Wunused-but-set-variable]
    double *Lx, *Lz ;
                 ^
In file included from ../Core/cholmod_change_factor.c:115:
../Core/t_cholmod_change_factor.c:37:18: warning: variable 'Lz' set but not used [-Wunused-but-set-variable]
    double *Lx, *Lz ;
                 ^
../Core/cholmod_change_factor.c:262:17: warning: variable 'xsize' set but not used [-Wunused-but-set-variable]
    Int nsuper, xsize, ssize ;
                ^
../Core/cholmod_change_factor.c:525:21: warning: implicit conversion from 'size_t' (aka 'unsigned long') to 'double' changes value from 18446744073709551615 to 18446744073709551616 [-Wimplicit-const-int-float-conversion]
                xlnz = MIN (xlnz, Size_max) ;
                       ~~~~~~~~~~~^~~~~~~~~
../Include/cholmod_internal.h:207:19: note: expanded from macro 'Size_max'
#define Size_max ((size_t) (-1))
   

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c ../Core/cholmod_common.c -o cholmod_common.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c ../Core/cholmod_complex.c -o cholmod_complex.o
clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -c ../Core/cholmod_copy.c -o cholmod_copy.o
clang -arch arm64 -I"/Library/Framew

../Core/cholmod_copy.c:275:21: warning: variable 'up' set but not used [-Wunused-but-set-variable]
    Int nrow, ncol, up, lo, values, diag, astype ;
                    ^
../Core/cholmod_copy.c:275:25: warning: variable 'lo' set but not used [-Wunused-but-set-variable]
    Int nrow, ncol, up, lo, values, diag, astype ;
                        ^
2 warnings generated.
In file included from ../Core/cholmod_dense.c:45:
../Core/t_cholmod_dense.c:27:23: warning: variable 'Az' set but not used [-Wunused-but-set-variable]
    double *Ax, *Xx, *Az, *Xz ;
                      ^
../Core/t_cholmod_dense.c:27:13: warning: variable 'Ax' set but not used [-Wunused-but-set-variable]
    double *Ax, *Xx, *Az, *Xz ;
            ^
../Core/t_cholmod_dense.c:27:28: warning: variable 'Xz' set but not used [-Wunused-but-set-variable]
    double *Ax, *Xx, *Az, *Xz ;
                           ^
In file included from ../Core/cholmod_dense.c:47:
../Core/t_cholmod_dense.c:27:28: warning: variable 'Xz' set but 

...

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -DDLONG -c  ../Supernodal/cholmod_super_solve.c -o cholmod_l_super_solve.o


4 warnings generated.
../Supernodal/cholmod_super_symbolic.c:562:7: warning: variable 'totsize' set but not used [-Wunused-but-set-variable]
                Int totsize ;
                    ^
../Supernodal/cholmod_super_symbolic.c:184:17: warning: unused variable 'env_max_fraction' [-Wunused-variable]
    const char* env_max_fraction;
                ^
../Supernodal/cholmod_super_symbolic.c:185:12: warning: unused variable 'max_fraction' [-Wunused-variable]
    double max_fraction;
           ^
../Supernodal/cholmod_super_symbolic.c:181:17: warning: unused variable 'env_use_gpu' [-Wunused-variable]
    const char* env_use_gpu;
                ^
../Supernodal/cholmod_super_symbolic.c:182:17: warning: unused variable 'env_max_bytes' [-Wunused-variable]
    const char* env_max_bytes;
                ^
../Supernodal/cholmod_super_symbolic.c:183:12: warning: unused variable 'max_bytes' [-Wunused-variable]
    size_t max_bytes;
           ^
../Supernodal/cholmod_super_symbolic.c:577:24: war

clang -arch arm64 -I"/Library/Frameworks/R.framework/Resources/include" -DNDEBUG -I../../AMD/Include -I../../AMD/Source -I../../COLAMD/Include -I../Include -I../../SuiteSparse_config -DNPARTITION -DNPRINT  -I/opt/R/arm64/include    -fPIC  -falign-functions=64 -Wall -g -O2  -DDLONG -c  ../Supernodal/cholmod_super_symbolic.c -o cholmod_l_super_symbolic.o
ar -rucs ../../CHOLMOD.a cholmod_aat.o cholmod_add.o cholmod_band.o cholmod_change_factor.o cholmod_common.o cholmod_complex.o cholmod_copy.o cholmod_dense.o cholmod_error.o cholmod_factor.o cholmod_memory.o cholmod_sparse.o cholmod_transpose.o cholmod_triplet.o cholmod_version.o cholmod_check.o cholmod_read.o cholmod_write.o cholmod_amd.o cholmod_analyze.o cholmod_colamd.o cholmod_etree.o cholmod_factorize.o cholmod_postorder.o cholmod_rcond.o cholmod_resymbol.o cholmod_rowcolcounts.o cholmod_rowfac.o cholmod_solve.o cholmod_spsolve.o cholmod_drop.o cholmod_horzcat.o cholmod_norm.o cholmod_scale.o cholmod_sdmult.o cholmod_ssmult.o cholm

ld: warning: directory not found for option '-L/opt/gfortran/lib/gcc/aarch64-apple-darwin20.0/12.2.0'
ld: warning: directory not found for option '-L/opt/gfortran/lib'
ld: library not found for -lgfortran
clang-14: error: linker command failed with exit code 1 (use -v to see invocation)
make: *** [/Library/Frameworks/R.framework/Resources/share/make/shlib.mk:10: Matrix.so] Error 1
ERROR: compilation failed for package ‘Matrix’
* removing ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library/Matrix’
ERROR: dependency ‘Matrix’ is not available for package ‘lme4’
* removing ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library/lme4’
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/private/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T/RtmpcKOUEV/downloaded_packages’
R[write to console]: 
R[write to console]: 



PackageNotInstalledError: The R package "lme4" is not installed.

# GLMM

In [21]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Assuming df is your DataFrame
df = encoded

# Enable automatic conversion from pandas to R
pandas2ri.activate()

# Convert the DataFrame to an R object
rdf = pandas2ri.py2rpy(df)

# Import the necessary R packages
lme4 = importr('lme4')
base = importr('base')
Matrix = importr('Matrix')

# Define the model formula
# Replace this with your actual formula
formula = 'Signal ~ (1|Subject) + Deaf + dB_encoded + Frequency_8000 + Frequency_16000 + Frequency_24000 + Frequency_32000'

# Fit the model
fit = lme4.glmer(formula, data=rdf, family=ro.r('binomial'))

# Print the summary
print(base.summary(fit))


PackageNotInstalledError: The R package "lme4" is not installed.